<a href="https://colab.research.google.com/github/elylaila/seminario/blob/main/Explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/lcl23-xnlm-lab/blob/main/notebooks/2_Attribution_Contrastive.ipynb)

In [1]:
%%capture
# Run in Colab to install local packages
%pip install ferret-xai bitsandbytes accelerate numpy==1.23.1
%pip install git+https://github.com/huggingface/transformers.git
# %pip install git+https://github.com/inseq-team/inseq.git

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pandas as pd
from ferret import Benchmark

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Seminario/

Mounted at /content/drive


In [20]:
name = 'dbmdz/bert-base-italian-cased' # inserire bert-ita
model = AutoModelForSequenceClassification.from_pretrained('final_model/checkpoint-4375') # inserire path modello finale
tokenizer = AutoTokenizer.from_pretrained(name)
print(model.config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1'}


In [15]:
predictions = pd.read_csv('final_model_predictions.csv', usecols=['y_pred', 'y_true'])
predictions

,y_true,y_pred
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
995,0,0
996,0,1
997,0,0
998,1,1


In [18]:
df = pd.read_csv('dataset/df_test_clean.csv').drop(columns=['homotransphobic'])
df = pd.concat([df, predictions], axis=1)
df

,text,rationales,y_true,y_pred
0,quasi quasi è meglio femminiello!,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",1,0
1,"sotto quel tweet, c'è un ammasso di ritardati ...",NaN,0,0
2,brava mara come sempre puntatona.. sei la zia ...,NaN,0,0
3,copio i gay cagatemi !!! se vi chiedessi di de...,NaN,0,0
4,io: è bono eh papà: ma non eri frocia te,NaN,0,0
...,...,...,...,...
995,rt quindi tu dai dei piglianculo alla gente e ...,NaN,0,0
996,quel sono a lavoro andré che suona un po come ...,NaN,0,1
997,115 gol e 48 assist in 283 presenze. nella top...,NaN,0,0
998,ma non rompermi le balle culattone di merda,"[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5...",1,1


In [26]:
df_errors = df[(df['y_true']==0) & (df['y_pred']==1)].drop(columns=['rationales'])
df_errors.shape

(53, 3)

In [27]:
df_errors_sample = df_errors.sample(n=5, random_state=42)
df_errors_sample

,text,y_true,y_pred
383,4) se un ragazzo desse un pugno in faccia ad u...,0,1
817,comunque siamo 5 froci in una piccola macchi a...,0,1
911,"aug 19 2022, 19:36: bernardino femminielli",0,1
292,io preferisco fidarmi da chi non nasconde il s...,0,1
853,"pensa , prima di diceva ricchione...",0,1


In [30]:
sentences = list(df_errors_sample.text)
sentences

['4) se un ragazzo desse un pugno in faccia ad una ragazza perché ha insultato uno con un "checca di merda" avrebbe non solo il mio appoggio ma anche la mia ammirazione.',
 'comunque siamo 5 froci in una piccola macchi a diretti verso una serata frocia',
 'aug 19 2022, 19:36: bernardino femminielli',
 'io preferisco fidarmi da chi non nasconde il suo volto ma come sempre la gente che pensa di fare il culattone col le terga dellaltro abbonda ¯\\ () /¯',
 'pensa , prima di diceva ricchione...']

In [ ]:
bench = Benchmark(model, tokenizer)
all_explanations = bench.explain(sentences[0], target=1)
bench.show_table(all_explanations)

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]